# GS Quantify 2019 - Pranav Hari IIT Madras - CE17B115

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import pandas as pd

## Loading Training Data

In [1]:
data = [line.strip() for line in open("train_set.txt", "r").readlines()]
data = [{"LineNumber": line.split('~')[0], "Text": line.split('~')[1]} for line in data]
data = pd.DataFrame(data)
data.head()

,LineNumber,Text
0,L0,gs.api.log.57_13-09-2019_06:59 34703 INFO gs.o...
1,L1,gs.compute.log.30_2019-09-12_07:04:00 39946 IN...
2,L2,gs.compute.log.77_2019-09-11_10:53:00 94359 IN...
3,L3,gs.compute.log.38_2019-09-01_13:31:00 80828 IN...
4,L4,"gs.api.log.38_Sep 01, 2019_13:20:00 44944 INFO..."


In [2]:
datat = [line.strip() for line in open("public_test_set.txt", "r").readlines()]
datat = [{"LineNumber": line.split('~')[0], "Text": line.split('~')[1]} for line in datat]
datat = pd.DataFrame(datat)
datat.shape

(702, 2)

In [3]:
datap = [line.strip() for line in open("private_test_set.txt", "r").readlines()]
datap = [{"LineNumber": line.split('~')[0], "Text": line.split('~')[1]} for line in datap]
datap = pd.DataFrame(datap)
datap.shape

(1050, 2)

## Feature Engineering

In [5]:
data['mod_txt'] = data['Text'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
datat['mod_txt'] = datat['Text'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
datap['mod_txt'] = datap['Text'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
data.head()

,LineNumber,Text,mod_txt
0,L0,gs.api.log.57_13-09-2019_06:59 34703 INFO gs.o...,gs.api.log._--_: INFO gs.openstack.wsgi.serve...
1,L1,gs.compute.log.30_2019-09-12_07:04:00 39946 IN...,gs.compute.log._--_:: INFO gs.openapi.server ...
2,L2,gs.compute.log.77_2019-09-11_10:53:00 94359 IN...,gs.compute.log._--_:: INFO gs.api.openstack.c...
3,L3,gs.compute.log.38_2019-09-01_13:31:00 80828 IN...,gs.compute.log._--_:: INFO gs.resource.claims...
4,L4,"gs.api.log.38_Sep 01, 2019_13:20:00 44944 INFO...","gs.api.log._Sep , _:: INFO gs.resource.claims..."


In [15]:
data['mod_txt2'] = data['mod_txt'].apply(lambda x: x.replace('_','').replace('-','').replace('  ','').replace(':','').replace('[','').replace(']',''))
datat['mod_txt2'] = datat['mod_txt'].apply(lambda x: x.replace('_','').replace('-','').replace('  ','').replace(':','').replace('[','').replace(']','') )
datap['mod_txt2'] = datap['mod_txt'].apply(lambda x: x.replace('_','').replace('-','').replace('  ','').replace(':','').replace('[','').replace(']','') )
data.head()

,LineNumber,Text,mod_txt,mod_txt2
0,L0,gs.api.log.57_13-09-2019_06:59 34703 INFO gs.o...,gs.api.log._--_: INFO gs.openstack.wsgi.serve...,gs.api.log.INFO gs.openstack.wsgi.server reqy...
1,L1,gs.compute.log.30_2019-09-12_07:04:00 39946 IN...,gs.compute.log._--_:: INFO gs.openapi.server ...,gs.compute.log.INFO gs.openapi.server reqfpkx...
2,L2,gs.compute.log.77_2019-09-11_10:53:00 94359 IN...,gs.compute.log._--_:: INFO gs.api.openstack.c...,gs.compute.log.INFO gs.api.openstack.compute.w...
3,L3,gs.compute.log.38_2019-09-01_13:31:00 80828 IN...,gs.compute.log._--_:: INFO gs.resource.claims...,gs.compute.log.INFO gs.resource.claims reqjyt...
4,L4,"gs.api.log.38_Sep 01, 2019_13:20:00 44944 INFO...","gs.api.log._Sep , _:: INFO gs.resource.claims...","gs.api.log.Sep , INFO gs.resource.claims reqf..."


In [0]:
def respext(texty):
  ind = len(texty.split('  ')) - 1
  try:
    return texty.split('  ')[ind]
  except:
    return texty.split('  ')[ind-1]

In [0]:
data['response'] = data['mod_txt2'].apply(lambda x: respext(x))
datat['response'] = datat['mod_txt2'].apply(lambda x: respext(x))
datap['response'] = datap['mod_txt2'].apply(lambda x: respext(x))

In [43]:
data.head()

,LineNumber,Text,mod_txt,mod_txt2,response,api_code
0,L0,gs.api.log.57_13-09-2019_06:59 34703 INFO gs.o...,gs.api.log._--_: INFO gs.openstack.wsgi.serve...,gs.api.log.INFO gs.openstack.wsgi.server reqy...,HTTP exception thrown No entry found for any e...,api
1,L1,gs.compute.log.30_2019-09-12_07:04:00 39946 IN...,gs.compute.log._--_:: INFO gs.openapi.server ...,gs.compute.log.INFO gs.openapi.server reqfpkx...,Creating event networkvifplugged for instance,compute
2,L2,gs.compute.log.77_2019-09-11_10:53:00 94359 IN...,gs.compute.log._--_:: INFO gs.api.openstack.c...,gs.compute.log.INFO gs.api.openstack.compute.w...,Creating event networkvifplugged for instance,compute
3,L3,gs.compute.log.38_2019-09-01_13:31:00 80828 IN...,gs.compute.log._--_:: INFO gs.resource.claims...,gs.compute.log.INFO gs.resource.claims reqjyt...,Creating event networkvifplugged for instance,compute
4,L4,"gs.api.log.38_Sep 01, 2019_13:20:00 44944 INFO...","gs.api.log._Sep , _:: INFO gs.resource.claims...","gs.api.log.Sep , INFO gs.resource.claims reqf...","""DELETE /v/lwjqtyfinpxu/servers/details HTTP/....",api


In [0]:
data['api_code'] = data['Text'].apply(lambda x:x.split('.')[1])
datat['api_code'] = datat['Text'].apply(lambda x:x.split('.')[1])
datap['api_code'] = datap['Text'].apply(lambda x:x.split('.')[1])

In [44]:
data['api_text'] = data['api_code']+'-'+data['response']
datat['api_text'] = datat['api_code']+'-'+datat['response']
datap['api_text'] = datap['api_code']+'-'+datap['response']
data.head()

,LineNumber,Text,mod_txt,mod_txt2,response,api_code,api_text
0,L0,gs.api.log.57_13-09-2019_06:59 34703 INFO gs.o...,gs.api.log._--_: INFO gs.openstack.wsgi.serve...,gs.api.log.INFO gs.openstack.wsgi.server reqy...,HTTP exception thrown No entry found for any e...,api,api-HTTP exception thrown No entry found for a...
1,L1,gs.compute.log.30_2019-09-12_07:04:00 39946 IN...,gs.compute.log._--_:: INFO gs.openapi.server ...,gs.compute.log.INFO gs.openapi.server reqfpkx...,Creating event networkvifplugged for instance,compute,compute-Creating event networkvifplugged for i...
2,L2,gs.compute.log.77_2019-09-11_10:53:00 94359 IN...,gs.compute.log._--_:: INFO gs.api.openstack.c...,gs.compute.log.INFO gs.api.openstack.compute.w...,Creating event networkvifplugged for instance,compute,compute-Creating event networkvifplugged for i...
3,L3,gs.compute.log.38_2019-09-01_13:31:00 80828 IN...,gs.compute.log._--_:: INFO gs.resource.claims...,gs.compute.log.INFO gs.resource.claims reqjyt...,Creating event networkvifplugged for instance,compute,compute-Creating event networkvifplugged for i...
4,L4,"gs.api.log.38_Sep 01, 2019_13:20:00 44944 INFO...","gs.api.log._Sep , _:: INFO gs.resource.claims...","gs.api.log.Sep , INFO gs.resource.claims reqf...","""DELETE /v/lwjqtyfinpxu/servers/details HTTP/....",api,"api-""DELETE /v/lwjqtyfinpxu/servers/details HT..."


## Universal Sentence Encoder - Text to Vector

In [58]:
with tf.Graph().as_default():
    embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
    messages = tf.placeholder(dtype=tf.string, shape=[None])
    output = embed(messages)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        train_embed = session.run(output, feed_dict={messages: data['mod_txt'].values})
        test_embed = session.run(output, feed_dict={messages: datat['mod_txt'].values})
        priv_embed = session.run(output, feed_dict={messages: datap['mod_txt'].values})

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
resmap = {}
for i in range(1,21):
    resmap.update({i-1:f'C{i}'})

## KMeans Clustering on the vector Space

In [0]:
kmeans = KMeans(n_clusters=10,init='k-means++',max_iter=500,n_init=10,random_state=0)
kmeans.fit(train_embed)
y = kmeans.predict(test_embed)
y_private = kmeans.predict(priv_embed)
results = pd.Series(y,index=None)
res_priv = pd.Series(y_private,index=None)
results = results.map(resmap)
res_priv = res_priv.map(resmap)
resdata = {'LineNumber':datat['LineNumber'],'Class':results}
respriv = {'LineNumber':datap['LineNumber'],'Class':res_priv}
df = pd.DataFrame(data=resdata)
df_p = pd.DataFrame(data=respriv)
df.to_csv('./tfidf12_use.csv',index=False)
df_p.to_csv('./priv_use_12.csv',index=False)

## Gaussian Mixture Models

In [53]:
gmm = GaussianMixture(n_components=20,covariance_type="tied",init_params='random',max_iter=100,n_init=15,random_state=0,verbose=10)
gmm.fit(train_embed)
y = gmm.predict(test_embed)
y_private = gmm.predict(priv_embed)
results = pd.Series(y,index=None)
res_priv = pd.Series(y_private,index=None)
results = results.map(resmap)
res_priv = res_priv.map(resmap)
resdata = {'LineNumber':datat['LineNumber'],'Class':results}
respriv = {'LineNumber':datap['LineNumber'],'Class':res_priv}
df = pd.DataFrame(data=resdata)
df_p = pd.DataFrame(data=respriv)
df.to_csv('./gmm10_use_apiresp.csv',index=False)
df_p.to_csv('./priv_use_apiresp_gmm10.csv',index=False)

Initialization 0
Initialization converged: True	 time lapse 94.66725s	 ll 1972.31062
Initialization 1
Initialization converged: True	 time lapse 93.61759s	 ll 1972.31062
Initialization 2
Initialization converged: True	 time lapse 93.28769s	 ll 1972.31062
Initialization 3
Initialization converged: True	 time lapse 92.82486s	 ll 1972.31062
Initialization 4
Initialization converged: True	 time lapse 92.05074s	 ll 1972.31062
Initialization 5
Initialization converged: True	 time lapse 91.44325s	 ll 1972.31062
Initialization 6
Initialization converged: True	 time lapse 91.02416s	 ll 1972.31062
Initialization 7
Initialization converged: True	 time lapse 91.04739s	 ll 1972.31062
Initialization 8
Initialization converged: True	 time lapse 90.60640s	 ll 1972.31062
Initialization 9
Initialization converged: True	 time lapse 91.10997s	 ll 1972.31062
Initialization 10
Initialization converged: True	 time lapse 90.75458s	 ll 1972.31062
Initialization 11
Initialization converged: True	 time lapse 90.

# Final Solution used was Kmeans Clustering on Universal Sentence Encoding Vector Space - no of clusters = 20, n_iterations = 500